In [27]:
from torch.autograd import Variable
from WSharingModel import *
from torch import nn, optim
from torch.nn import CrossEntropyLoss
from torch.nn import BCEWithLogitsLoss
import torch
from torchvision import datasets
from torch import optim
import dlc_practical_prologue as prologue
from AuxModel import *
from torch.autograd import Variable
from torch.nn import functional as F
from time import time
from SimpleCNN import *
from P2 import *
import math

In [28]:
def visualize_loss(model,loss_model,lr):
    plt.plot(loss_model[0],loss_model[1], color='orange')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Evolution of the loss during training with learning rate {:>5} of the model {:>5}.'.format(lr,model.__class__.__name__))
    plt.show()

In [29]:
def print_results(model,optimizer,learning_rate,NB_EPOCHS,MINI_BATCH_SIZE, train_input, train_target,test_input, test_target, time_diff):
     print('model: {:6}, optimizer: {:6}, learning rate: {:6}, num epochs: {:3}, '
                    'mini batch size: {:3}, train error: {:5.2f}%, test error: {:5.2f}%, time: {:.3f}'.format(
                    model.__class__.__name__,
                    optimizer.__name__,
                    learning_rate,
                    NB_EPOCHS,
                    MINI_BATCH_SIZE,
                    model.compute_nb_errors(model, train_input, train_target, MINI_BATCH_SIZE) / train_input.size(0) * 100,
                    model.compute_nb_errors(model, test_input, test_target, MINI_BATCH_SIZE) / test_input.size(0) * 100,
                    time_diff
                    )
                )

In [30]:
def generate_disc_data(n=1000):
    inputs = torch.rand(1000,2)
    distance = torch.norm((inputs - torch.Tensor([[0.5, 0.5]])), 2, 1, True)
    targets = distance.mul(math.sqrt(2*math.pi)).sub(1).sign().add(1).div(2).long()
    return inputs, targets

inputs, targets = generate_disc_data(n=1000)

def split_data(inputs, targets, train_part, val_part, test_part):
    """Splits dataset into training, validation and test set.
    
    Output:
    train-, validation- and test data  : (percentage * n)x2 dimension FloatTensor
    train-, validation- and test targets : (percentage * n)x1 dimension LongTensor
    """
    if (train_part + val_part + test_part > 1):
        raise ValueError('Requested data partitions too large.')

    training_size = math.floor(inputs.size()[0] * train_part)
    train_data = inputs.narrow(0, 0, training_size)
    train_targets = targets.narrow(0, 0, training_size)

    val_size = math.floor(inputs.size()[0] * val_part)
    validation_data = inputs.narrow(0, training_size, val_size)
    validation_targets = targets.narrow(0, training_size, val_size)

    test_size = math.floor(inputs.size()[0] * test_part)
    test_data = inputs.narrow(0, training_size+val_size, test_size)
    test_targets = targets.narrow(0, training_size+val_size, test_size)
    
    return train_data, train_targets, validation_data, validation_targets, test_data, test_targets
    
train_data, train_targets, validation_data, \
    validation_targets, test_data, test_targets = split_data(
inputs, targets, 0.7, 0.1, 0.2)
    
learning_rates = [0.001]

# normalize it
mean, std = train_data.mean(), train_data.std() 
train_data.sub_(mean).div_(std)
test_data.sub_(mean).div_(std)

train_data, train_targets, train_classes = Variable(train_data), Variable(train_targets), Variable(torch.Tensor([0,1]))
test_data, test_targets, test_classes = Variable(test_data), Variable(test_targets), Variable(torch.Tensor([0,1]))

In [25]:
 def calculate_results(result_dct):
    k = 0
    for j in range(1):
        for i in range(len(learning_rates)):
            models = [P2()]
            model = models[j]
            optimizer = op(model.parameters(),lr = learning_rates[i])
            if (j == 0):
                start_time = time()
                loss_aux = train_model_P2(model, optimizer,  train_data, train_targets, NB_EPOCHS, MINI_BATCH_SIZE,losses[0])
                end_time = time()
            elif (j<2):
                #print("here")
                start_time = time()
                loss_aux = train_model_WS(model, optimizer,  train_data, train_targets, NB_EPOCHS, MINI_BATCH_SIZE,losses[0])
                end_time = time()
            elif (j==4):
                start_time = time()
                loss_aux = train_model(model, optimizer,  train_data, train_targets, NB_EPOCHS, MINI_BATCH_SIZE,losses[0])
                end_time = time()
            elif(j>1):
                #print("here here")
                start_time = time()
                loss_aux = train_model_AM(model, optimizer,  train_data, train_targets, train_classes,NB_EPOCHS, MINI_BATCH_SIZE,losses[0],alpha,beta)
                end_time = time()
            #print("model:",model)
            time_diff = end_time - start_time
            print_results(model,op,learning_rates[i],NB_EPOCHS,MINI_BATCH_SIZE, train_data, train_targets,test_data, test_targets, time_diff)
            print(result_dct[k])
            train_error = model.compute_nb_errors(model, train_data, train_targets, MINI_BATCH_SIZE) / train_data.size(0) * 100
            test_error = model.compute_nb_errors(model, test_data, test_targets, MINI_BATCH_SIZE) / test_data.size(0) * 100
            result_dct[k]['train_error'].append(train_error)
            result_dct[k]['test_error'].append(test_error)
            result_dct[k]['time'].append(time_diff)
            k += 1
    return result_dct
            #visualize_loss(model,loss_aux,learning_rates[i])

In [31]:
from collections import defaultdict
result_dct = defaultdict(dict)
for i in range(10):
    result_dct[i]['train_error'] = []
    result_dct[i]['test_error'] = []
    result_dct[i]['time'] = []
for i in range(10):
    
    train_data, train_targets, train_classes, test_data, test_targets, test_classes = prologue.generate_pair_sets(nb=1000)
    # normalize it
    mean, std = train_data.mean(), train_data.std() 
    train_data.sub_(mean).div_(std)
    test_data.sub_(mean).div_(std)

    train_data, train_targets, train_classes = Variable(train_data), Variable(train_targets), Variable(train_classes)
    test_data, test_targets = Variable(test_data), Variable(test_targets)
    
    print(result_dct)
    result_dct = calculate_results(result_dct)

defaultdict(<class 'dict'>, {0: {'train_error': [], 'test_error': [], 'time': []}, 1: {'train_error': [], 'test_error': [], 'time': []}, 2: {'train_error': [], 'test_error': [], 'time': []}, 3: {'train_error': [], 'test_error': [], 'time': []}, 4: {'train_error': [], 'test_error': [], 'time': []}, 5: {'train_error': [], 'test_error': [], 'time': []}, 6: {'train_error': [], 'test_error': [], 'time': []}, 7: {'train_error': [], 'test_error': [], 'time': []}, 8: {'train_error': [], 'test_error': [], 'time': []}, 9: {'train_error': [], 'test_error': [], 'time': []}})


NameError: name 'SimpleModel' is not defined